In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from matplotlib import pyplot
from math import sqrt
from PIL import Image
import os

from pgan import PGAN, WeightedSum
from tensorflow.keras import backend

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
# Create a Keras callback that periodically saves generated images and updates alpha in WeightedSum layers
class GANMonitor(keras.callbacks.Callback):
  def __init__(self, num_img=16, latent_dim=512, prefix=''):
    self.num_img = num_img
    self.latent_dim = latent_dim
    self.random_latent_vectors = tf.random.normal(shape=[num_img, self.latent_dim], seed=9434)
    self.steps_per_epoch = 0
    self.epochs = 0
    self.steps = self.steps_per_epoch * self.epochs
    self.n_epoch = 0
    self.prefix = prefix
  
  def set_prefix(self, prefix=''):
    self.prefix = prefix
  
  def set_steps(self, steps_per_epoch, epochs):
    self.steps_per_epoch = steps_per_epoch
    self.epochs = epochs
    self.steps = self.steps_per_epoch * self.epochs

  def on_epoch_begin(self, epoch, logs=None):
    self.n_epoch = epoch


  def on_epoch_end(self, epoch, logs=None):
    samples = self.model.generator(self.random_latent_vectors)
    samples = (samples * 0.5) + 0.5
    n_grid = int(sqrt(self.num_img))

    fig, axes = pyplot.subplots(n_grid, n_grid, figsize=(4*n_grid, 4*n_grid))
    sample_grid = np.reshape(samples[:n_grid * n_grid], (n_grid, n_grid, samples.shape[1], samples.shape[2], samples.shape[3]))

    for i in range(n_grid):
      for j in range(n_grid):
        axes[i][j].set_axis_off()
        samples_grid_i_j = Image.fromarray((sample_grid[i][j] * 255).astype(np.uint8))
        samples_grid_i_j = samples_grid_i_j.resize((128,128))
        axes[i][j].imshow(np.array(samples_grid_i_j))
    title = f'images/plot_{self.prefix}_{epoch:05d}.png'
    pyplot.savefig(title, bbox_inches='tight')
    print(f'\n saved {title}')
    pyplot.close(fig)
  

  def on_batch_begin(self, batch, logs=None):
    # Update alpha in WeightedSum layers
    alpha = ((self.n_epoch * self.steps_per_epoch) + batch) / float(self.steps - 1)
    #print(f'\n {self.steps}, {self.n_epoch}, {self.steps_per_epoch}, {alpha}')
    for layer in self.model.generator.layers:
      if isinstance(layer, WeightedSum):
        backend.set_value(layer.alpha, alpha)
    for layer in self.model.discriminator.layers:
      if isinstance(layer, WeightedSum):
        backend.set_value(layer.alpha, alpha)

In [3]:

# DEFINE FILEPATH AND PARAMETERS
# can use celeb A mask dataset on https://github.com/switchablenorms/CelebAMask-HQ 
DATA_ROOT = '/root/dataset/haseeb_data/dataset_AD/dataset' 
NOISE_DIM = 512
# Set the number of batches, epochs and steps for trainining.
# Look 800k images(16x50x1000) per each lavel
BATCH_SIZE = [16, 16, 16, 16, 16, 16, 4]
EPOCHS = 50
#STEPS_PER_EPOCH = 308 #1000

In [4]:
# Normalilze [-1, 1] input images
def preprocessing_image(img):
  img = img.astype('float32')
  img = (img - 127.5) / 127.5
  return img

train_image_generator = ImageDataGenerator(horizontal_flip=True, preprocessing_function=preprocessing_image)
train_dataset = train_image_generator.flow_from_directory(batch_size=BATCH_SIZE[0],
                                                          directory=DATA_ROOT,
                                                          shuffle=True,
                                                          target_size=(4,4),
                                                          class_mode='binary')

STEPS_PER_EPOCH = train_dataset.n//train_dataset.batch_size

Found 2429 images belonging to 1 classes.


In [5]:
# Instantiate the optimizer for both networks
# learning_rate will be equalized per each layers by the WeightScaling scheme
generator_optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.0, beta_2=0.99, epsilon=1e-8)
discriminator_optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.0, beta_2=0.99, epsilon=1e-8)



cbk = GANMonitor(num_img=64, latent_dim=NOISE_DIM, prefix='0_init')
cbk.set_steps(steps_per_epoch=STEPS_PER_EPOCH, epochs=EPOCHS)

# Instantiate the PGAN(PG-GAN) model.
pgan = PGAN(
    latent_dim = NOISE_DIM, 
    d_steps = 1,
)

checkpoint_path = f"ckpts/pgan_{cbk.prefix}.ckpt"

# Compile models
pgan.compile(
    d_optimizer=discriminator_optimizer,
    g_optimizer=generator_optimizer,
)

print(len(train_dataset))
# Draw models
tf.keras.utils.plot_model(pgan.generator, to_file=f'generator_{pgan.n_depth}.png', show_shapes=True)
tf.keras.utils.plot_model(pgan.discriminator, to_file=f'discriminator_{pgan.n_depth}.png', show_shapes=True)

# Start training the initial generator and discriminator
pgan.fit(train_dataset, steps_per_epoch = STEPS_PER_EPOCH, epochs = EPOCHS, callbacks=[cbk])
pgan.save_weights(checkpoint_path)

Model: "generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 512)]             0         
_________________________________________________________________
pixel_normalization (PixelNo (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 8192)              4194304   
_________________________________________________________________
weight_scaling_4 (WeightScal (None, 8192)              0         
_________________________________________________________________
bias_4 (Bias)                (None, 8192)              8192      
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 8192)              0         
_________________________________________________________________
pixel_normalization_1 (Pixel (None, 8192)              0 

Epoch 23/50
151/151 [==============================] - ETA: 0s - d_loss: -0.1558 - g_loss: 0.2856
 saved images/plot_0_init_00022.png
151/151 [==============================] - 8s 53ms/step - d_loss: -0.1559 - g_loss: 0.2873
Epoch 24/50
151/151 [==============================] - ETA: 0s - d_loss: -0.1683 - g_loss: 0.3537
 saved images/plot_0_init_00023.png
151/151 [==============================] - 8s 53ms/step - d_loss: -0.1675 - g_loss: 0.3544
Epoch 25/50
151/151 [==============================] - ETA: 0s - d_loss: -0.1607 - g_loss: 0.2726
 saved images/plot_0_init_00024.png
151/151 [==============================] - 8s 53ms/step - d_loss: -0.1609 - g_loss: 0.2727
Epoch 26/50
151/151 [==============================] - ETA: 0s - d_loss: -0.1721 - g_loss: 0.3348
 saved images/plot_0_init_00025.png
151/151 [==============================] - 8s 54ms/step - d_loss: -0.1720 - g_loss: 0.3349
Epoch 27/50
151/151 [==============================] - ETA: 0s - d_loss: -0.1518 - g_loss: 0.2415
 s

In [6]:
# Train faded-in / stabilized generators and discriminators
for n_depth in range(1, 7):
  # Set current level(depth)
  pgan.n_depth = n_depth

  # Set parameters like epochs, steps, batch size and image size
  steps_per_epoch = STEPS_PER_EPOCH
  epochs = int(EPOCHS*(BATCH_SIZE[0]/BATCH_SIZE[n_depth]))
  DATA_ROOT = f'/root/dataset/haseeb_data/dataset_AD/dataset-{n_depth}'
  train_dataset = train_image_generator.flow_from_directory(batch_size=BATCH_SIZE[n_depth],
                                                          directory=DATA_ROOT,
                                                          shuffle=True,
                                                          target_size=(4*(2**n_depth), 4*(2**n_depth)),
                                                          class_mode='binary')
  cbk.set_prefix(prefix=f'{n_depth}_fade_in')
  cbk.set_steps(steps_per_epoch=steps_per_epoch, epochs=epochs)

  # Put fade in generator and discriminator
  pgan.fade_in_generator()
  pgan.fade_in_discriminator()

  # Draw fade in generator and discriminator
  tf.keras.utils.plot_model(pgan.generator, to_file=f'generator_{n_depth}_fade_in.png', show_shapes=True)
  tf.keras.utils.plot_model(pgan.discriminator, to_file=f'discriminator_{n_depth}_fade_in.png', show_shapes=True)

  pgan.compile(
      d_optimizer=discriminator_optimizer,
      g_optimizer=generator_optimizer,
  )
  # Train fade in generator and discriminator
  pgan.fit(train_dataset, steps_per_epoch = steps_per_epoch, epochs = epochs, callbacks=[cbk])
  # Save models
  checkpoint_path = f"ckpts/pgan_{cbk.prefix}.ckpt"
  pgan.save_weights(checkpoint_path)

  # Change to stabilized generator and discriminator
  cbk.set_prefix(prefix=f'{n_depth}_stabilize')
  pgan.stabilize_generator()
  pgan.stabilize_discriminator()

  # Draw stabilized generator and discriminator
  tf.keras.utils.plot_model(pgan.generator, to_file=f'generator_{n_depth}_stabilize.png', show_shapes=True)
  tf.keras.utils.plot_model(pgan.discriminator, to_file=f'discriminator_{n_depth}_stabilize.png', show_shapes=True)
  pgan.compile(
      d_optimizer=discriminator_optimizer,
      g_optimizer=generator_optimizer,
  )
  # Train stabilized generator and discriminator
  pgan.fit(train_dataset, steps_per_epoch = steps_per_epoch, epochs = epochs, callbacks=[cbk])
  # Save models
  checkpoint_path = f"ckpts/pgan_{cbk.prefix}.ckpt"
  pgan.save_weights(checkpoint_path)

Found 2429 images belonging to 1 classes.
Model: "generator"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 512)]        0                                            
__________________________________________________________________________________________________
pixel_normalization (PixelNorma (None, 512)          0           input_2[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 8192)         4194304     pixel_normalization[0][0]        
__________________________________________________________________________________________________
weight_scaling_4 (WeightScaling (None, 8192)         0           dense_1[0][0]                    
________________________________________________

Epoch 1/50
151/151 [==============================] - ETA: 0s - d_loss: -0.6176 - g_loss: 1.4606
 saved images/plot_1_fade_in_00000.png
151/151 [==============================] - 14s 92ms/step - d_loss: -0.6237 - g_loss: 1.4656
Epoch 2/50
151/151 [==============================] - ETA: 0s - d_loss: -1.5256 - g_loss: 1.0558
 saved images/plot_1_fade_in_00001.png
151/151 [==============================] - 13s 89ms/step - d_loss: -1.5229 - g_loss: 1.0468
Epoch 3/50
151/151 [==============================] - ETA: 0s - d_loss: -1.4561 - g_loss: 0.8678
 saved images/plot_1_fade_in_00002.png
151/151 [==============================] - 14s 92ms/step - d_loss: -1.4569 - g_loss: 0.8705
Epoch 4/50
151/151 [==============================] - ETA: 0s - d_loss: -1.3039 - g_loss: 0.9837
 saved images/plot_1_fade_in_00003.png
151/151 [==============================] - 17s 113ms/step - d_loss: -1.3004 - g_loss: 0.9830
Epoch 5/50
151/151 [==============================] - ETA: 0s - d_loss: -0.9483 - g_los

Epoch 37/50
151/151 [==============================] - ETA: 0s - d_loss: -0.0816 - g_loss: 0.2932
 saved images/plot_1_fade_in_00036.png
151/151 [==============================] - 14s 92ms/step - d_loss: -0.0820 - g_loss: 0.2909
Epoch 38/50
151/151 [==============================] - ETA: 0s - d_loss: -0.0759 - g_loss: 0.2010
 saved images/plot_1_fade_in_00037.png
151/151 [==============================] - 14s 93ms/step - d_loss: -0.0765 - g_loss: 0.2062
Epoch 39/50
151/151 [==============================] - ETA: 0s - d_loss: -0.0841 - g_loss: 0.3044
 saved images/plot_1_fade_in_00038.png
151/151 [==============================] - 14s 93ms/step - d_loss: -0.0845 - g_loss: 0.3087
Epoch 40/50
151/151 [==============================] - ETA: 0s - d_loss: -0.0601 - g_loss: 0.1305
 saved images/plot_1_fade_in_00039.png
151/151 [==============================] - 14s 92ms/step - d_loss: -0.0594 - g_loss: 0.1269
Epoch 41/50
151/151 [==============================] - ETA: 0s - d_loss: -0.0522 - g

Epoch 1/50
151/151 [==============================] - ETA: 0s - d_loss: -0.0598 - g_loss: 0.1555
 saved images/plot_1_stabilize_00000.png
151/151 [==============================] - 14s 90ms/step - d_loss: -0.0608 - g_loss: 0.1532
Epoch 2/50
151/151 [==============================] - ETA: 0s - d_loss: -0.0489 - g_loss: 0.1906
 saved images/plot_1_stabilize_00001.png
151/151 [==============================] - 14s 90ms/step - d_loss: -0.0495 - g_loss: 0.1924
Epoch 3/50
151/151 [==============================] - ETA: 0s - d_loss: -0.0576 - g_loss: 0.2412
 saved images/plot_1_stabilize_00002.png
151/151 [==============================] - 14s 91ms/step - d_loss: -0.0583 - g_loss: 0.2402
Epoch 4/50
151/151 [==============================] - ETA: 0s - d_loss: -0.0370 - g_loss: 0.2116
 saved images/plot_1_stabilize_00003.png
151/151 [==============================] - 14s 90ms/step - d_loss: -0.0376 - g_loss: 0.2130
Epoch 5/50
151/151 [==============================] - ETA: 0s - d_loss: -0.0436 

Epoch 37/50
151/151 [==============================] - ETA: 0s - d_loss: -0.0270 - g_loss: 0.0641
 saved images/plot_1_stabilize_00036.png
151/151 [==============================] - 14s 91ms/step - d_loss: -0.0276 - g_loss: 0.0636
Epoch 38/50
151/151 [==============================] - ETA: 0s - d_loss: -0.0201 - g_loss: 0.1048
 saved images/plot_1_stabilize_00037.png
151/151 [==============================] - 14s 90ms/step - d_loss: -0.0206 - g_loss: 0.1056
Epoch 39/50
151/151 [==============================] - ETA: 0s - d_loss: -0.0252 - g_loss: 0.0912
 saved images/plot_1_stabilize_00038.png
151/151 [==============================] - 14s 90ms/step - d_loss: -0.0247 - g_loss: 0.0886
Epoch 40/50
151/151 [==============================] - ETA: 0s - d_loss: -0.0299 - g_loss: 0.1111
 saved images/plot_1_stabilize_00039.png
151/151 [==============================] - 14s 91ms/step - d_loss: -0.0299 - g_loss: 0.1104
Epoch 41/50
151/151 [==============================] - ETA: 0s - d_loss: -0.

...

Epoch 36/200
151/151 [==============================] - ETA: 0s - d_loss: -8.0645 - g_loss: 12.9937
 saved images/plot_6_stabilize_00035.png
151/151 [==============================] - 83s 553ms/step - d_loss: -8.0937 - g_loss: 12.8534
Epoch 37/200
151/151 [==============================] - ETA: 0s - d_loss: -6.8260 - g_loss: 9.6489
 saved images/plot_6_stabilize_00036.png
151/151 [==============================] - 83s 550ms/step - d_loss: -6.8409 - g_loss: 9.7328
Epoch 38/200
151/151 [==============================] - ETA: 0s - d_loss: -8.0257 - g_loss: 12.5052
 saved images/plot_6_stabilize_00037.png
151/151 [==============================] - 83s 552ms/step - d_loss: -8.0312 - g_loss: 12.3203
Epoch 39/200
151/151 [==============================] - ETA: 0s - d_loss: -7.9625 - g_loss: 10.9504
 saved images/plot_6_stabilize_00038.png
151/151 [==============================] - 83s 550ms/step - d_loss: -8.0691 - g_loss: 10.9990
Epoch 40/200
151/151 [==============================] - ETA: 0

Epoch 71/200
151/151 [==============================] - ETA: 0s - d_loss: -9.0602 - g_loss: 13.0599
 saved images/plot_6_stabilize_00070.png
151/151 [==============================] - 84s 553ms/step - d_loss: -9.0359 - g_loss: 12.9187
Epoch 72/200
151/151 [==============================] - ETA: 0s - d_loss: -8.6284 - g_loss: 11.5567
 saved images/plot_6_stabilize_00071.png
151/151 [==============================] - 84s 553ms/step - d_loss: -8.6607 - g_loss: 11.5724
Epoch 73/200
151/151 [==============================] - ETA: 0s - d_loss: -8.8503 - g_loss: 12.5943
 saved images/plot_6_stabilize_00072.png
151/151 [==============================] - 83s 551ms/step - d_loss: -8.9419 - g_loss: 12.6561
Epoch 74/200
151/151 [==============================] - ETA: 0s - d_loss: -8.6577 - g_loss: 12.0871
 saved images/plot_6_stabilize_00073.png
151/151 [==============================] - 84s 554ms/step - d_loss: -8.6505 - g_loss: 12.0380
Epoch 75/200
151/151 [==============================] - ETA:

Epoch 106/200
151/151 [==============================] - ETA: 0s - d_loss: -9.7363 - g_loss: 13.8044
 saved images/plot_6_stabilize_00105.png
151/151 [==============================] - 84s 554ms/step - d_loss: -9.6976 - g_loss: 13.8993
Epoch 107/200
151/151 [==============================] - ETA: 0s - d_loss: -9.4453 - g_loss: 12.7275
 saved images/plot_6_stabilize_00106.png
151/151 [==============================] - 83s 551ms/step - d_loss: -9.5011 - g_loss: 12.6223
Epoch 108/200
151/151 [==============================] - ETA: 0s - d_loss: -9.7149 - g_loss: 13.2002
 saved images/plot_6_stabilize_00107.png
151/151 [==============================] - 83s 550ms/step - d_loss: -9.6775 - g_loss: 12.6908
Epoch 109/200
151/151 [==============================] - ETA: 0s - d_loss: -9.4233 - g_loss: 15.0231
 saved images/plot_6_stabilize_00108.png
151/151 [==============================] - 83s 553ms/step - d_loss: -9.4040 - g_loss: 15.0145
Epoch 110/200
151/151 [==============================] -

Epoch 141/200
151/151 [==============================] - ETA: 0s - d_loss: -9.9814 - g_loss: 13.4544 
 saved images/plot_6_stabilize_00140.png
151/151 [==============================] - 83s 552ms/step - d_loss: -9.9270 - g_loss: 13.4251
Epoch 142/200
151/151 [==============================] - ETA: 0s - d_loss: -10.2488 - g_loss: 14.7333
 saved images/plot_6_stabilize_00141.png
151/151 [==============================] - 84s 553ms/step - d_loss: -10.2823 - g_loss: 14.8150
Epoch 143/200
151/151 [==============================] - ETA: 0s - d_loss: -9.9200 - g_loss: 14.1581 
 saved images/plot_6_stabilize_00142.png
151/151 [==============================] - 83s 551ms/step - d_loss: -9.8138 - g_loss: 14.2876
Epoch 144/200
151/151 [==============================] - ETA: 0s - d_loss: -10.2515 - g_loss: 13.6420
 saved images/plot_6_stabilize_00143.png
151/151 [==============================] - 83s 552ms/step - d_loss: -10.2398 - g_loss: 13.4872
Epoch 145/200
151/151 [===========================

Epoch 176/200
151/151 [==============================] - ETA: 0s - d_loss: -10.1930 - g_loss: 14.3130
 saved images/plot_6_stabilize_00175.png
151/151 [==============================] - 113s 749ms/step - d_loss: -10.2791 - g_loss: 14.4106
Epoch 177/200
151/151 [==============================] - ETA: 0s - d_loss: -9.9848 - g_loss: 13.4523
 saved images/plot_6_stabilize_00176.png
151/151 [==============================] - 134s 890ms/step - d_loss: -9.9847 - g_loss: 13.3941
Epoch 178/200
151/151 [==============================] - ETA: 0s - d_loss: -10.4097 - g_loss: 14.5153
 saved images/plot_6_stabilize_00177.png
151/151 [==============================] - 83s 553ms/step - d_loss: -10.3170 - g_loss: 14.4839
Epoch 179/200
151/151 [==============================] - ETA: 0s - d_loss: -11.0207 - g_loss: 13.5644
 saved images/plot_6_stabilize_00178.png
151/151 [==============================] - 84s 553ms/step - d_loss: -10.8665 - g_loss: 13.0922
Epoch 180/200
151/151 [=========================